In [ ]:
# This takes hours to download
# !wget https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z

In [ ]:
!7z x stackoverflow.com-Posts.7z

In [ ]:
from lxml import etree
from tqdm import tqdm

# count element for displaying progress bar
context = etree.iterparse('Posts.xml')
post_count = 0
for ev, elem in context:
    post_count+=1
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]
post_count

In [5]:
%%writefile dump.py
import sqlite3
from lxml import etree
from tqdm import tqdm

TABLE_CREATE_STRING = """
CREATE TABLE IF NOT EXISTS Post(
    AcceptedAnswerId INT,
    Title TEXT,
    Body TEXT,
    OwnerUserId INT,
    ClosedDate DATETIME,
    LastEditorUserId INT,
    PostTypeId INT,
    AnswerCount INT,
    OwnerDisplayName TEXT,
    CreationDate DATETIME,
    ViewCount INT,
    CommunityOwnedDate DATETIME,
    LastEditorDisplayName TEXT,
    FavoriteCount INT,
    Score INT,
    Id INT UNIQUE PRIMARY KEY,
    Tags TEXT,
    LastEditDate INT,
    LastActivityDate DATETIME,
    ParentId INT,
    CommentCount INT
)
"""
DB_FILE = 'so.db'
conn = sqlite3.connect(DB_FILE)
conn.execute(TABLE_CREATE_STRING)
conn.commit()
from lxml import etree
from tqdm import tqdm

# count element for displaying progress bar
context = etree.iterparse('Posts.xml')
total = 46947634
counter = 0
fields = []
buffer = []
insert_str = '''
    INSERT INTO Post (
        AcceptedAnswerId,
        Title,
        Body,
        OwnerUserId,
        ClosedDate,
        LastEditorUserId,
        PostTypeId,
        AnswerCount,
        OwnerDisplayName,
        CreationDate,
        ViewCount,
        CommunityOwnedDate,
        LastEditorDisplayName,
        FavoriteCount,
        Score,
        Id,
        Tags,
        LastEditDate,
        LastActivityDate,
        ParentId,
        CommentCount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''
def save_buffer( insert_str, buff, conn):
    conn.executemany(insert_str, buff)
    conn.commit()
for ev, elem in tqdm(context, total=total):
    counter+=1

    
    values = [
        elem.get("AcceptedAnswerId"),
        elem.get("Title"),
        elem.get("Body"),
        elem.get("OwnerUserId"),
        elem.get("ClosedDate"),
        elem.get("LastEditorUserId"),
        elem.get("PostTypeId"),
        elem.get("AnswerCount"),
        elem.get("OwnerDisplayName"),
        elem.get("CreationDate"),
        elem.get("ViewCount"),
        elem.get("CommunityOwnedDate"),
        elem.get("LastEditorDisplayName"),
        elem.get("FavoriteCount"),
        elem.get("Score"),
        elem.get("Id"),
        elem.get("Tags"),
        elem.get("LastEditDate"),
        elem.get("LastActivityDate"),
        elem.get("ParentId"),
        elem.get("CommentCount")
    ]
    buffer.append(values)
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]
    
    # save every 100k iterations
    if counter%100000==0:
        save_buffer(insert_str, buffer, conn)
        buffer = []
    
    
save_buffer(insert_str, buffer, conn)
conn.close()

Overwriting dump.py


In [6]:
!python dump.py

100%|############################| 46947634/46947634 [30:00<00:00, 26077.71it/s]
